<a href="https://colab.research.google.com/github/Kartikey0412/google_colab/blob/main/cnn_lstm_gru_intro_imdb.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%matplotlib inline
import keras
import numpy as np
from keras import backend as K
from keras.utils.data_utils import get_file
from keras.utils import np_utils
from keras.utils.np_utils import to_categorical
from keras.models import Sequential, Model
from keras.layers import Input, Embedding, Reshape, merge, LSTM, Bidirectional
from keras.layers import TimeDistributed, Activation, SimpleRNN, GRU
from keras.layers.core import Flatten, Dense, Dropout, Lambda
from keras.regularizers import l2, l1
from keras.layers.normalization import BatchNormalization
from keras.layers.pooling import *
from keras.optimizers import SGD, RMSprop, Adam
from keras.metrics import categorical_crossentropy, categorical_accuracy
from keras.layers.convolutional import *
from keras.preprocessing import image, sequence
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.image import ImageDataGenerator
from keras.applications.resnet50 import ResNet50
from keras.applications.vgg16 import VGG16
from keras.applications.inception_v3 import InceptionV3
from keras import applications

In [2]:
from keras.datasets import imdb as keras_imdb

In [3]:
(x_train, y_train), (x_test, y_test) = keras_imdb.load_data(num_words=None,
                                                       skip_top=0,
                                                       maxlen=None,
                                                       seed=113,
                                                       start_char=1,
                                                       oov_char=2,
                                                       index_from=3)

17465344/17464789 [==============================] - 0s 0us/step


<string>:6: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/datasets/imdb.py:159: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  x_train, y_train = np.array(xs[:idx]), np.array(labels[:idx])
/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/datasets/imdb.py:160: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 

In [ ]:
print(x_train[0:5][:10])
print(len(x_train[0]))

[list([1, 14, 22, 16, 43, 530, 973, 1622, 1385, 65, 458, 4468, 66, 3941, 4, 173, 36, 256, 5, 25, 100, 43, 838, 112, 50, 670, 22665, 9, 35, 480, 284, 5, 150, 4, 172, 112, 167, 21631, 336, 385, 39, 4, 172, 4536, 1111, 17, 546, 38, 13, 447, 4, 192, 50, 16, 6, 147, 2025, 19, 14, 22, 4, 1920, 4613, 469, 4, 22, 71, 87, 12, 16, 43, 530, 38, 76, 15, 13, 1247, 4, 22, 17, 515, 17, 12, 16, 626, 18, 19193, 5, 62, 386, 12, 8, 316, 8, 106, 5, 4, 2223, 5244, 16, 480, 66, 3785, 33, 4, 130, 12, 16, 38, 619, 5, 25, 124, 51, 36, 135, 48, 25, 1415, 33, 6, 22, 12, 215, 28, 77, 52, 5, 14, 407, 16, 82, 10311, 8, 4, 107, 117, 5952, 15, 256, 4, 31050, 7, 3766, 5, 723, 36, 71, 43, 530, 476, 26, 400, 317, 46, 7, 4, 12118, 1029, 13, 104, 88, 4, 381, 15, 297, 98, 32, 2071, 56, 26, 141, 6, 194, 7486, 18, 4, 226, 22, 21, 134, 476, 26, 480, 5, 144, 30, 5535, 18, 51, 36, 28, 224, 92, 25, 104, 4, 226, 65, 16, 38, 1334, 88, 12, 16, 283, 5, 16, 4472, 113, 103, 32, 15, 16, 5345, 19, 178, 32])
 list([1, 194, 1153, 194, 825

In [4]:
word_to_id = keras.datasets.imdb.get_word_index()

1646592/1641221 [==============================] - 0s 0us/step


In [5]:
word_to_id["horrible"]

524

In [6]:
list(word_to_id.items())[:4]

[('fawn', 34701), ('tsukino', 52006), ('nunnery', 52007), ('sonja', 16816)]

In [7]:
# Add terms to our vocabulary
word_to_id = {word : (word_id + 3) for word, word_id in keras.datasets.imdb.get_word_index().items()}  # Add an offset of 3 to leave room for the new terms
word_to_id["<PAD>"] = 0
word_to_id["<START>"] = 1
word_to_id["<UNK>"] = 2

# Also define the opposite mapping
id_to_word = {word_id: word for word, word_id in word_to_id.items()}

In [8]:
def get_review(x, i):
    return " ".join(id_to_word[id_] for id_ in x[i])

def get_label(y, i):
    if y[i] == 1:
        return "Positive"
    else:
        return "Negative"

In [ ]:
for review_index in range(10):
    review = get_review(x_train, review_index)
    label = get_label(y_train, review_index)
    print("*" * 50)
    print(f"Review index: {review_index}")
    print(f"Label: {label}")
    print("-" * 50)
    print(review)
    print("*" * 50)

**************************************************
Review index: 0
Label: Positive
--------------------------------------------------
<START> this film was just brilliant casting location scenery story direction everyone's really suited the part they played and you could just imagine being there robert redford's is an amazing actor and now the same being director norman's father came from the same scottish island as myself so i loved the fact there was a real connection with this film the witty remarks throughout the film were great it was just brilliant so much that i bought the film as soon as it was released for retail and would recommend it to everyone to watch and the fly fishing was amazing really cried at the end it was so sad and you know what they say if you cry at a film it must have been good and this definitely was also congratulations to the two little boy's that played the part's of norman and paul they were just brilliant children are often left out of the praising list 

## Data Prep

In [9]:
vocab_size = 5000
trn = [np.array([i if i<vocab_size-1 else vocab_size-1 for i in s]) for s in x_train]
test = [np.array([i if i<vocab_size-1 else vocab_size-1 for i in s]) for s in x_test]

In [ ]:
lens = np.array(list(map(len, trn)))
print('Maximum text length:', lens.max(),' -- Minimum length:', lens.min(), '-- Mean length of text:',lens.mean())

Maximum text length: 2494  -- Minimum length: 11 -- Mean length of text: 238.71364


### Padding

In [10]:
# we'll pad all inputs to obtain homogeneous inputs of dim 500
seq_len = 500

trn = sequence.pad_sequences(trn, maxlen = seq_len,value=0, padding="post", truncating="post")
test = sequence.pad_sequences(test, maxlen = seq_len,value=0, padding="post", truncating="post")

In [ ]:
print(trn.shape)
get_review(trn, 1)

(25000, 500)


"<START> big hair big orange bad music and a giant safety orange these are the words to best describe this terrible movie i love cheesy horror movies and i've seen hundreds but this had got to be on of the worst ever made the plot is paper thin and ridiculous the acting is an orange the script is completely laughable the best is the end showdown with the cop and how he worked out who the killer is it's just so damn terribly written the clothes are orange and funny in equal orange the hair is big lots of orange orange men wear those cut orange orange that show off their orange orange that men actually wore them and the music is just orange trash that plays over and over again in almost every scene there is trashy music orange and orange taking away bodies and the orange still doesn't close for orange all orange aside this is a truly bad film whose only charm is to look back on the disaster that was the 80's and have a good old laugh at how bad everything was back then <PAD> <PAD> <PAD> 

# Modeling 

## MLP

In [ ]:
def MLP():
    model = Sequential()
    model.add(Embedding(vocab_size,32,input_length=seq_len))
    
    model.add(Flatten())
    model.add(Dense(100,activation='relu'))
    model.add(BatchNormalization())
    model.add(Dropout(0.5))
    
    model.add(Dense(100,activation='relu'))
    model.add(BatchNormalization())
    model.add(Dropout(0.5))
    
    model.add(Dense(1,activation='sigmoid'))

    model.compile(loss='binary_crossentropy',optimizer=Adam(),metrics=['accuracy'])
    print(model.summary())
    
    return model

model = MLP()  

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 500, 32)           160000    
_________________________________________________________________
flatten (Flatten)            (None, 16000)             0         
_________________________________________________________________
dense (Dense)                (None, 100)               1600100   
_________________________________________________________________
batch_normalization (BatchNo (None, 100)               400       
_________________________________________________________________
dropout (Dropout)            (None, 100)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 100)               10100     
_________________________________________________________________
batch_normalization_1 (Batch (None, 100)               4

In [ ]:
model.fit(trn, y_train, validation_data=(test, y_test), epochs=10, batch_size=512)

Epoch 1/10
49/49 [==============================] - 2s 37ms/step - loss: 0.1479 - accuracy: 0.9478 - val_loss: 0.4276 - val_accuracy: 0.8331
Epoch 2/10
49/49 [==============================] - 2s 35ms/step - loss: 0.0757 - accuracy: 0.9771 - val_loss: 0.3874 - val_accuracy: 0.8302
Epoch 3/10
49/49 [==============================] - 2s 35ms/step - loss: 0.0420 - accuracy: 0.9898 - val_loss: 0.3637 - val_accuracy: 0.8397
Epoch 4/10
49/49 [==============================] - 2s 35ms/step - loss: 0.0208 - accuracy: 0.9964 - val_loss: 0.3996 - val_accuracy: 0.8268
Epoch 5/10
49/49 [==============================] - 2s 36ms/step - loss: 0.0141 - accuracy: 0.9972 - val_loss: 0.4357 - val_accuracy: 0.8366
Epoch 6/10
49/49 [==============================] - 2s 36ms/step - loss: 0.0079 - accuracy: 0.9990 - val_loss: 0.4691 - val_accuracy: 0.8368
Epoch 7/10
49/49 [==============================] - 2s 36ms/step - loss: 0.0064 - accuracy: 0.9991 - val_loss: 0.5350 - val_accuracy: 0.8359
Epoch 8/10
49

In [ ]:
scores = model.evaluate(test,y_test,verbose=0)
print('loss: ', scores[0],'- accuracy: ', scores[1])

loss:  0.6699978113174438 - accuracy:  0.8358799815177917


## 1D CNN

In [ ]:
def CNN():
    model = Sequential()
    model.add(Embedding(vocab_size, 32, input_length=seq_len))
    
    #model.add(Dropout(0.2))
    model.add(Conv1D(64, 5, padding='same', activation='relu'))
    model.add(Dropout(0.2))
    model.add(MaxPooling1D())
    
    model.add(Flatten())
    
    model.add(Dense(100, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(1, activation='sigmoid'))
    
    model.compile(loss='binary_crossentropy',optimizer=Adam(),metrics=['accuracy'])
    print(model.summary())
    
    return model

model = CNN()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 500, 32)           160000    
_________________________________________________________________
conv1d (Conv1D)              (None, 500, 64)           10304     
_________________________________________________________________
dropout_2 (Dropout)          (None, 500, 64)           0         
_________________________________________________________________
max_pooling1d (MaxPooling1D) (None, 250, 64)           0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 16000)             0         
_________________________________________________________________
dense_3 (Dense)              (None, 100)               1600100   
_________________________________________________________________
dropout_3 (Dropout)          (None, 100)              

In [ ]:
model.fit(trn, y_train, validation_data=(test, y_test), epochs=4)

Epoch 1/4
782/782 [==============================] - 13s 10ms/step - loss: 0.5232 - accuracy: 0.6959 - val_loss: 0.2769 - val_accuracy: 0.8844
Epoch 2/4
782/782 [==============================] - 8s 10ms/step - loss: 0.2123 - accuracy: 0.9218 - val_loss: 0.2812 - val_accuracy: 0.8819
Epoch 3/4
782/782 [==============================] - 8s 10ms/step - loss: 0.1555 - accuracy: 0.9444 - val_loss: 0.2964 - val_accuracy: 0.8835
Epoch 4/4
782/782 [==============================] - 8s 10ms/step - loss: 0.1108 - accuracy: 0.9621 - val_loss: 0.3689 - val_accuracy: 0.8780


In [ ]:
scores = model.evaluate(test,y_test,verbose=0)
print('loss: ', scores[0],'- accuracy: ', scores[1])

loss:  0.36885955929756165 - accuracy:  0.8780400156974792


## LSTM

In [12]:
def RNN_LSTM():
    model = Sequential()
    model.add(Embedding(vocab_size,5,input_length=seq_len))
    model.add(LSTM(50))
    model.add(Dropout(0.25))
    model.add(Dense(1,activation='sigmoid'))

    model.compile(loss='binary_crossentropy',optimizer=Adam(),metrics=['accuracy'])
    print(model.summary())
    
    return model

model = RNN_LSTM() 

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 500, 5)            25000     
_________________________________________________________________
lstm (LSTM)                  (None, 50)                11200     
_________________________________________________________________
dropout (Dropout)            (None, 50)                0         
_________________________________________________________________
dense (Dense)                (None, 1)                 51        
Total params: 36,251
Trainable params: 36,251
Non-trainable params: 0
_________________________________________________________________
None


In [13]:
model.fit(trn, y_train, validation_data=(test, y_test), epochs=4, batch_size=512)

Epoch 1/4
49/49 [==============================] - 11s 69ms/step - loss: 0.6932 - accuracy: 0.4938 - val_loss: 0.6931 - val_accuracy: 0.5000
Epoch 2/4
49/49 [==============================] - 3s 56ms/step - loss: 0.6929 - accuracy: 0.5038 - val_loss: 0.6931 - val_accuracy: 0.5000
Epoch 3/4
49/49 [==============================] - 3s 56ms/step - loss: 0.7082 - accuracy: 0.5121 - val_loss: 0.6938 - val_accuracy: 0.5070
Epoch 4/4
49/49 [==============================] - 3s 56ms/step - loss: 0.6924 - accuracy: 0.5144 - val_loss: 0.6929 - val_accuracy: 0.5056


In [14]:
scores = model.evaluate(test,y_test,verbose=0, batch_size=512)
print('loss: ', scores[0],'- accuracy: ', scores[1])

loss:  0.6929231286048889 - accuracy:  0.5055599808692932


## GRU

In [15]:
def RNN_GRU():
    model = Sequential()
    model.add(Embedding(vocab_size,5,input_length=seq_len))
    model.add(GRU(50))
    model.add(Dropout(0.25))
    model.add(Dense(1,activation='sigmoid'))

    model.compile(loss='binary_crossentropy',optimizer=Adam(),metrics=['accuracy'])
    print(model.summary())
    
    return model

model = RNN_GRU()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 500, 5)            25000     
_________________________________________________________________
gru (GRU)                    (None, 50)                8550      
_________________________________________________________________
dropout_1 (Dropout)          (None, 50)                0         
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 51        
Total params: 33,601
Trainable params: 33,601
Non-trainable params: 0
_________________________________________________________________
None


In [16]:
model.fit(trn, y_train, validation_data=(test, y_test), epochs=4, batch_size=512)

Epoch 1/4
49/49 [==============================] - 4s 63ms/step - loss: 0.6931 - accuracy: 0.5000 - val_loss: 0.6931 - val_accuracy: 0.5050
Epoch 2/4
49/49 [==============================] - 3s 55ms/step - loss: 0.6929 - accuracy: 0.5030 - val_loss: 0.6930 - val_accuracy: 0.5050
Epoch 3/4
49/49 [==============================] - 3s 55ms/step - loss: 0.6924 - accuracy: 0.5107 - val_loss: 0.6929 - val_accuracy: 0.5052
Epoch 4/4
49/49 [==============================] - 3s 55ms/step - loss: 0.6917 - accuracy: 0.5101 - val_loss: 0.6934 - val_accuracy: 0.5001


In [17]:
scores = model.evaluate(test,y_test,verbose=0)
print('loss: ', scores[0],'- accuracy: ', scores[1])

loss:  0.693374752998352 - accuracy:  0.5001199841499329
